In [2]:
import numpy as np
from scipy.linalg import lstsq, pinv, norm
from scipy.io import loadmat
from scipy.io.wavfile import write
from scipy.sparse import csr_matrix
from numpy.random import permutation

Apply *bpIRLS* and *lassoADMM* functions to recover a highly sub-sampled audio signal, which is an example of compressed sensing (CS). CS relies on two key facts:
1. The signal must be sparse in some known basis, i.e., for a signal $s \in R^{D}$ that we wish to recover, we have that
\begin{equation*}
    s = \sum_{i = 1}^{D} u_{i} x_{i} = Ux
\end{equation*}
has $x_{i} \neq 0$ for $k << D$ values of $i$. This implies that the signal $s$ is **compressible**.
2. We sample $s$ by obtaining linear measurements of the form
\begin{equation*}
    y_{i} = \langle a_{i}, s\rangle \qquad i = 1,\dots,N,
\end{equation*}
which in matrix form implies
\begin{equation*}
    y = As = AUx = Bx,
\end{equation*}
where $B = AU \in R^{N \times D}$. The above is an underdetermined system for $N < D$, so traditional theory tells us that we cannot recover $x$ unless we take at least $D$ measurements. However, if $x$ has only $k$ nonzero entries, compressed sensing tells us that we only need $N = O(k)$ measurements to recover $x$, and consequently $s$, exactly. It turns out that the way to recover $x$ exactly is to solve a sparse regression problem!

In this script, we have the basis $U$ already. Our measurement matrix is designed to sample $p = 456$ random locations of $s$. The size of $s$ is $D = 319,725$.

**Goal:** Recover $s$ using standard least-squares regression by minimizing $\left\|Bx - y\right\|_{2}$.

In [10]:
import helperFunctions as hf

In [7]:
# load basis and true signal
data = loadmat("pianoBasis")
U = data['pianoBasis']
fs = data['fs'][0][0]

data = loadmat("marySong")
s = data['marySong']
D = len(s)

# play true signal - listen to the wav file
write('marySong.wav', fs, s)

# sample s at random locations
p = 456                                                 # Number of samples
idx = np.sort(np.random.permutation(D)[:p])             # index of samples
Rmeas = csr_matrix((np.ones(p), (np.arange(p), idx)), shape=(p,D))   # Measurement matrix

# measurement matrix and measurements
B = Rmeas @ U.T
y = Rmeas @ s
xtrue = U @ s

## Recover s using l2 minimization
xhat = lstsq(B.A,y)[0]
s_l2 = U.T @ xhat
write('marySong_l2.wav', fs, s_l2)
err = norm(xhat - xtrue)/norm(xtrue)
print(err)

0.9988125458894361


Recovering $s$ using **standard least-squares regression**:

minimizing $\left\| Bx - y\right\|_2$

yields what sounds mostly like static, with very little similarity to the original song.

In [11]:
## Recover s using IRLS
for p in [0.01, 0.1, 1]:
    xhat = hf.BP_irls(B.A,y,p)
    s_irls = U.T @ xhat
    fileName = 'marySong_bpIRLS'+str(p)+'.wav'
    write(fileName, fs, s_irls)
    err = norm(xhat - xtrue)/norm(xtrue)
    print('p = %.2f, err = %.2f'%(p, err))

AttributeError: module 'helperFunctions' has no attribute 'BP_irls'

Recovering $s$ using **IRLS for sparse regression**:

$\min_{x \in R^D} \left\|x\right\|_{p}$ subject to $Bx = y$

gives much better results. With low values of $p$ (0.01, 0.1) there are a few errant notes, but with $p = 1$, the recovered signal sounds almost exactly like the original.

In [ ]:
## Recover s using ADMM
for lam in [0.01, 0.1, 1]:
    xhat = hf.lassoADMM(B.A,y,lam,maxIter=100)
    s_admm = U.T @ xhat
    fileName = 'marySong_lassoADMM'+str(lam)+'.wav'
    write(fileName, fs, s_admm)
    err = norm(xhat - xtrue)/norm(xtrue)
    print('$\lambda$ = %.2f, err = %.2f'%(p, err))

Recovering $s$ using **LASSO-ADMM**:

$\min_{x \in R^{D}} \frac{1}{2} \left\|Ax - b\right\|_{2}^{2} + \frac{\lambda}{2} \left\|x\right\|_{1}$

also yields good results. Low values of $\lambda \leq 0.5$ give the best recovery, while larger values have more inaudible or missing notes. 